<a href="https://colab.research.google.com/github/kmachida12345/openai-playground/blob/main/slack_bot_socket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler
import openai

SLACK_APP_TOKEN = os.environ.get("SLACK_APP_TOKEN")
SLACK_BOT_TOKEN = os.environ.get("SLACK_BOT_TOKEN")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

openai.api_key = OPENAI_API_KEY

app = App(token=SLACK_BOT_TOKEN)

@app.event("app_mention")                                                                                                   
def command_handler(body, say):
    text = body['event'].get('text')
    thread_ts = body['event']['ts']
    
    say(text="wait a minute please...", thread_ts=thread_ts)
    
    response = chat_with_gpt(text)
    say(text=response, thread_ts=thread_ts)

def chat_with_gpt(prompt):
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    message = response.choices[0].message.content
    return message

if __name__ == "__main__":
    handler = SocketModeHandler(app, SLACK_APP_TOKEN)
    handler.start()


めも
AWS LambdaにデプロイするときはSlackBoltのアダプタがあるのでそれを使う。OpenAiのAPIはレスポンスに時間がかかるのでSlack側がタイムアウトして再度リクエストが飛んでくる。API呼び出しと返信処理をSQSにキューイングしてやればとりあえず回避できる。受けるイベントがメンションじゃなければlazy_listenerが使えるかもだけど試してない。